In [16]:
# Import open3d

import open3d as o3d
# point_cloud_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/data_odometry_velodyne/dataset/sequences/21_onecar_pcd/velodyne"
# pc_id = 380 
# bb_id = 380- 361
# # Load point cloud
# # point_cloud = o3d.io.read_point_cloud(point_cloud_path+ "/" + "000423.pcd")
# # point_cloud = o3d.io.read_point_cloud(point_cloud_path+ "/" + "000361.pcd")
# point_cloud = o3d.io.read_point_cloud(point_cloud_path+ "/" + "000380.pcd")
# print("Point cloud loaded")

hatchback_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/sequence1/669254000.pcd"
point_cloud = o3d.io.read_point_cloud(hatchback_path)
# 

# View point cloud
o3d.visualization.draw_geometries([point_cloud])



In [2]:
# Load bounding box information file for all the frames
# Path: /home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/kiss-icp/python/results/21_onecar_odom/velodyne_poses_tum.txt
'''
bb_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/OpenPCDet/tools/bb_pose.txt"
bb_poses = []
with open(bb_path, 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        # Convert to float
        line = [float(i) for i in line]
        bb_poses.append(line)
# print(bb_poses)

print("bb_poses", bb_poses[bb_id])
'''

IndexError: list index out of range

In [18]:
''' 
[ 1.6157433e+01  4.2528806e+00 -8.7529200e-01  4.2412167e+00
   1.6887267e+00  1.5699031e+00 -1.3737211e-02]
'''
import numpy as np
bb_box = np.array([7.5243578e+00,  6.9556141e+00, -1.0197808e+00  ,3.8175061e+00
,   1.6549100e+00  ,1.6159281e+00,  1.6904783e-02])


In [19]:
import numpy as np
# Bounding Box Pose : Height, Width, Length, X, Y, Z, Yaw,
def compute_bounding_box_object(X, Y, Z, Length, Width, Height, Yaw):
    # Create the bounding box vertices in its local frame
    # vertices = np.array([
    #     [Length / 2, 0, Width / 2],
    #     [Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, Width / 2],
    #     [Length / 2, Height, Width / 2],
    #     [Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, Width / 2]
    # ])

    vertices = np.array([
        [Length / 2, Width / 2, -Height/2],
        [Length / 2, -Width / 2, -Height/2],
        [-Length / 2, -Width / 2, -Height/2],
        [-Length / 2, Width / 2, -Height/2],
        [Length / 2, Width / 2, Height/2],
        [Length / 2, -Width / 2, Height/2],
        [-Length / 2, -Width / 2, Height/2],
        [-Length / 2, Width / 2, Height/2]

    ])

    # Create the rotation matrix based on Yaw
    rotation_matrix = np.array([
        [np.cos(Yaw), 0, np.sin(Yaw)],
        [0, 1, 0],
        [-np.sin(Yaw), 0, np.cos(Yaw)]
    ])

    # Rotate and translate the bounding box to its final pose
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    translated_vertices = rotated_vertices + np.array([X, Y, Z])

    return o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(translated_vertices))

def compute_bounding_box_edges(X, Y, Z, Length, Width, Height,Yaw ):
    # Create the bounding box vertices in its local frame
    # vertices = np.array([
    #     [Length / 2, 0, Width / 2],
    #     [Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, Width / 2],
    #     [Length / 2, Height, Width / 2],
    #     [Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, Width / 2]
    # ])

    vertices = np.array([
        [Length / 2, Width / 2, -Height/2],
        [Length / 2, -Width / 2, -Height/2],
        [-Length / 2, -Width / 2, -Height/2],
        [-Length / 2, Width / 2, -Height/2],
        [Length / 2, Width / 2, Height/2],
        [Length / 2, -Width / 2, Height/2],
        [-Length / 2, -Width / 2, Height/2],
        [-Length / 2, Width / 2, Height/2]

    ])
    # Create the rotation matrix based on Yaw
    rotation_matrix = np.array([
        [np.cos(Yaw), 0, np.sin(Yaw)],
        [0, 1, 0],
        [-np.sin(Yaw), 0, np.cos(Yaw)]
    ])

    # Rotate and translate the bounding box to its final pose
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    translated_vertices = rotated_vertices + np.array([X, Y, Z])

        # Define the edges of the bounding box
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]
    # Create a Homogenous transformation matrix
    T = np.eye(4)
    T[:3, :3] = rotation_matrix
    T[:3, 3] = np.array([X, Y, Z])

    return o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(translated_vertices),
        lines=o3d.utility.Vector2iVector(edges)
    ), T
    # return o3d.utility.Vector3dVector(translated_vertices)

# Create a bbox object
bb_object = compute_bounding_box_object(*bb_box)
bb_edges, T = compute_bounding_box_edges(*bb_box)
print("bb_edges", bb_edges)
print("bb_object", type(bb_object))
o3d.visualization.draw_geometries([point_cloud, bb_edges])

# Visualize bb_edges
# o3d.visualization.draw_geometries([bb_object])

bb_edges LineSet with 12 lines.
bb_object <class 'open3d.cuda.pybind.geometry.OrientedBoundingBox'>


In [12]:
car_object = point_cloud.crop(bb_object)
# car_object_canonical = car_object

# Copy the point cloud to a new variable by just copying the data
import copy
car_object_frame = copy.deepcopy(car_object)
car_object_frame.transform(np.linalg.inv(T))
# print(type(car_object))
# add coordinate frame
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
# Save the cropped point cloud to a file
o3d.io.write_point_cloud("hatch_back_car.pcd", car_object)
o3d.io.write_point_cloud("hatch_back_in_object_frame.pcd", car_object_frame)

o3d.visualization.draw_geometries([car_object_frame, coordinate_frame])


In [7]:
# Read cropped_car.pcd and visualize it
car = o3d.io.read_point_cloud("cropped_car.pcd")
print("car", car)
# add coordinate frame
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
# visualize
vis = o3d.visualization.Visualizer()
o3d.visualization.draw_geometries([car, coordinate_frame])
# convert to numpy array
car_np = np.asarray(car.points)
print(car_np.shape)

car PointCloud with 587 points.
(587, 3)


In [6]:
# Run clustering on point cloud data
import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler



# Convert the point cloud to a NumPy array
points = np.asarray(car_object.points)

# Normalize the points (optional but recommended for DBSCAN)
scaler = StandardScaler()
normalized_points = scaler.fit_transform(points)

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.4, min_samples=10)  # Adjust eps and min_samples as per your data
labels = dbscan.fit_predict(normalized_points)

# Create a new point cloud with cluster labels as colors
colored_point_cloud = o3d.geometry.PointCloud()
colored_point_cloud.points = o3d.utility.Vector3dVector(points)
colored_point_cloud.colors = o3d.utility.Vector3dVector(np.random.random((len(points), 3)))

# Assign cluster labels to the colors
for i, label in enumerate(labels):
    if label == -1:  # Noise points will be shown in black
        colored_point_cloud.colors[i] = [0, 0, 0]
    else:
        # Assign random colors to clusters
        np.random.seed(label)
        colored_point_cloud.colors[i] = np.random.random(3)


# Visualize the clustered point cloud
o3d.visualization.draw_geometries([colored_point_cloud])
